## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import os
import sys
sys.path.append(os.path.join(os.path.dirname(sys.path[0])))
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
#file_to_load=os.path.join(os.getcwd(), "WeatherPy_Database.csv")
file_to_load="../Weather_Database/WeatherPy_Database.csv"
#"#/Users/Mark/Desktop/Analysis Projects/World_Weather_Analysis/#
city_data_df = pd.read_csv(file_to_load)


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your destination?"))
max_temp=float(input("What is the maximum temperature you would like for your destination?"))

What is the minimum temperature you would like for your destination?65
What is the maximum temperature you would like for your destination?77


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                     (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
1,1,Bengkulu,-3.80,102.27,74.91,91,100,7.34,ID,overcast clouds,2020-12-16 05:35:03
10,10,Menongue,-14.66,17.69,66.78,56,71,6.02,AO,broken clouds,2020-12-16 05:37:51
14,14,Pinotepa Nacional,16.32,-98.02,70.66,84,61,2.21,MX,broken clouds,2020-12-16 05:37:51
15,15,Hilo,19.73,-155.09,73.40,83,1,2.86,US,light rain,2020-12-16 05:34:29
24,24,Mayumba,-3.43,10.66,76.33,92,100,7.09,GA,heavy intensity rain,2020-12-16 05:37:53


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        2
Description    0
Date           0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferable_cities=preferred_cities_df.dropna()

In [10]:
preferable_cities.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        0
Description    0
Date           0
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df =preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,74.91,overcast clouds,-3.80,102.27,
10,Menongue,AO,66.78,broken clouds,-14.66,17.69,
14,Pinotepa Nacional,MX,70.66,broken clouds,16.32,-98.02,
15,Hilo,US,73.40,light rain,19.73,-155.09,
24,Mayumba,GA,76.33,heavy intensity rain,-3.43,10.66,
31,Puerto Ayora,EC,71.01,overcast clouds,-0.74,-90.35,
46,Vaini,TO,77.00,broken clouds,-21.20,-175.20,
50,Guatire,VE,68.00,overcast clouds,10.48,-66.54,
51,Tingi,TZ,73.31,broken clouds,-11.30,35.03,
53,Boqueirao,BR,72.37,overcast clouds,-7.50,-36.13,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
counters=1
record_count=1
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels=requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"]=hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [13]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,74.91,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
10,Menongue,AO,66.78,broken clouds,-14.66,17.69,Ritz Lauca
14,Pinotepa Nacional,MX,70.66,broken clouds,16.32,-98.02,Hotel Maria Del Carmen
15,Hilo,US,73.40,light rain,19.73,-155.09,Hilo Hawaiian Hotel
24,Mayumba,GA,76.33,heavy intensity rain,-3.43,10.66,LIKWALE LODGE
...,...,...,...,...,...,...,...
393,Byron Bay,AU,75.20,broken clouds,-28.65,153.62,Backpackers Inn on the Beach
394,Namibe,AO,66.69,scattered clouds,-15.20,12.15,Hotel Chik Chik Namibe
397,Kahului,US,75.20,few clouds,20.89,-156.47,Maui Seaside Hotel
399,Santiago,CL,66.00,clear sky,-33.46,-70.65,Hotel Panamericano


In [14]:
hotel_df.dropna()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,74.91,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
10,Menongue,AO,66.78,broken clouds,-14.66,17.69,Ritz Lauca
14,Pinotepa Nacional,MX,70.66,broken clouds,16.32,-98.02,Hotel Maria Del Carmen
15,Hilo,US,73.40,light rain,19.73,-155.09,Hilo Hawaiian Hotel
24,Mayumba,GA,76.33,heavy intensity rain,-3.43,10.66,LIKWALE LODGE
...,...,...,...,...,...,...,...
393,Byron Bay,AU,75.20,broken clouds,-28.65,153.62,Backpackers Inn on the Beach
394,Namibe,AO,66.69,scattered clouds,-15.20,12.15,Hotel Chik Chik Namibe
397,Kahului,US,75.20,few clouds,20.89,-156.47,Maui Seaside Hotel
399,Santiago,CL,66.00,clear sky,-33.46,-70.65,Hotel Panamericano


In [15]:
hotel_df.isnull().sum()

City           0
Country        2
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [16]:
#7. Drop all of the empty rows in hotel_df
clean_hotel_df=hotel_df.dropna()

In [17]:
clean_hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [18]:
# 8a. Create the output File (CSV)
#output_data_file= "Vacation_Search/WeatherPy_vacation"
output_data_file=os.path.join(os.getcwd(),"WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
#clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Call the figure to plot the data.
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))